In [1]:
import socket
import time
import threading
import xml.etree.ElementTree as ET
import json
import base64
import dash
from dash import html, dcc, Input, Output, State, callback_context
import dash_daq as daq
from dash.exceptions import PreventUpdate
from collections import deque
import plotly.graph_objs as go


X = deque(maxlen=20)
X.append(1)
Y_1 = deque(maxlen=20)
Y_1.append(1)
Y_2 = deque(maxlen=20)
Y_2.append(1)

# Global variables
monitor_thread_running = False
client_socket = None
switch_states = [False] * 8  # Tüm anahtarlar başlangıçta kapalı
response_str_lists = {i: [] for i in range(23, 31)}

def process_toggleSwitch_config(toggleSwitch_params):
    toggleSwitch = daq.BooleanSwitch(
        id=toggleSwitch_params.get('id', 'switch-1'),
        on=False,
        label=toggleSwitch_params.get('label', 'label-1'),
        labelPosition=toggleSwitch_params.get('labelPosition', 'top')
    )
    return toggleSwitch

def process_graph_config(graph_params_list):
    traces = []
    for graph_params in graph_params_list:
        x = graph_params.get('x_values', [])
        y = graph_params.get('y_values', [])
        trace = dcc.Graph(
            id=graph_params.get('id', 'custom-graph'),
            figure={'data': [{'x': x, 'y': y, 'type': 'scatter', 'mode': 'lines', 'name': graph_params.get('title', 'Custom Graph')}],
                    'layout': {'title': graph_params.get('title', 'Custom Graph')}}
        )
        traces.append(trace)
    return traces

def process_thermometer_config(thermometer_params):
    thermometer = daq.Thermometer(
        id=thermometer_params.get('id', 'thermometer'),
        value=float(thermometer_params.get('value', 0)),
        height=int(thermometer_params.get('height', 192)),
        width=int(thermometer_params.get('width', 20)),
        color=thermometer_params.get('color', '#FF0000'),
        min=float(thermometer_params.get('min', 0)),
        max=float(thermometer_params.get('max', 100)),
        base=int(thermometer_params.get('base', 10)),
        logarithmic=thermometer_params.get('logarithmic', False),
        showCurrentValue=thermometer_params.get('showCurrentValue', True),
        units=thermometer_params.get('units', 'C'),
        theme=thermometer_params.get('theme', 'dark'),
        label=thermometer_params.get('label', ''),
        labelPosition=thermometer_params.get('labelPosition', 'top'),
        scale=thermometer_params.get('scale', ''),
        className=thermometer_params.get('className', ''),
        style=thermometer_params.get('style', {})
    )
    return thermometer

def process_gauge_config(gauge_params):
    gauge = daq.Gauge(
        id=gauge_params.get('id', 'my-gauge'),
        value=float(gauge_params.get('value', 0)),
        min=float(gauge_params.get('min', 0)),
        max=float(gauge_params.get('max', 100)),
        color=gauge_params.get('color', '#9B51E0'),
        label=gauge_params.get('label', 'Default')
    )
    print("process_gauge_config function is called.")
    return gauge

def parse_xml_config(xml_string):
    root = ET.fromstring(xml_string)
    config = {}

    for component in root.findall('Component'):
        component_name = component.get('name')
        params = {}
        for param in component.findall('Parameter'):
            param_name = param.get('name')
            param_value = param.text
            param_type = param.get('type')
            if param_type == 'string':
                params[param_name] = param_value
            elif param_type == 'boolean':
                params[param_name] = param_value.lower() == 'true'
        
        if component_name not in config:
            config[component_name] = []
        config[component_name].append(params)

    return config





def main(server_ip, server_port):
    global client_socket
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect((server_ip, server_port))
        print("Connection established.")
    except Exception as e:
        print("Error:", e)
        print("Failed to establish connection with the server.")

app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True 

app.layout = html.Div(
    children=[
        html.Div(
            id="baslik",
            children=[html.H1("STM32 CONTROL PANEL", style={'color': '#00008B', 'text-align': 'center'})],
            style={'width': '1350px', 'height': '60px', 'background-color': '#DDF3F6', 'margin-bottom': '10px'}
        ),
        html.Div(
            children=[
                html.Div(id='sol_div',
                         children=[
                             html.Label("IP ADRESİ", style={'color': "#fec036", "margin-left": "20px", "margin-bottom": "10px"}),
                             html.Label("PORT", style={'color': "#fec036", "margin-left": "110px", "margin-right": "30px", "margin-bottom": "15px"}),
                             dcc.Input(id='ip-input', type='text', value='', style={'color': "#fec036", 'height': '25px', 'width': '150px', "margin-left": "5px"}),
                             dcc.Input(id='port-input', type='number', value='', style={'color': "#fec036", 'height': '25px', 'width': '157px', "margin-left": "5px", 'margin-top': '10px'})
                         ],
                         style={'background-color':'#CCE5FF', 'width': '400px', "height": "60px", 'display': 'inline-block', "margin-top": "20px"}),
                html.Div([
                    html.Button('Connect', id='connect-button', n_clicks=0, style={"color": "#fec036", 'height': '30px', 'width': '250px', "margin-left": "5px", "margin-top": "20px"}),
                    html.Button('Disconnect', id='disconnect-button', n_clicks=0, style={"color": "#fec036", 'height': '30px', 'width': '250px', "margin-left": "5px", "margin-top": "20px"}),
                    dcc.Upload(
                        id='upload-xml-2',
                        children=html.Div(['Drag and Drop or ', html.A('Select a File')]),
                        style={ 'width': '100%',
                                'height': '60px',
                                'lineHeight': '60px',
                                'borderWidth': '1px',
                                'borderStyle':                                'dashed',
                                'borderRadius': '5px',
                                'textAlign': 'center',
                                'margin': '10px'
                        },
                        multiple=False
                    ),
                    html.Div(id='output-container-2', style={'margin-top': '20px'}),
                    
                ],
                style={'background-color': '#DDF3F6', 'width': '400px', "height": "35px", 'display': 'inline-block'}),
                html.Div(id='output-state')
            ],
            style={'width': '450px', 'height': '1488px', 'background-color': '#DDF3F6', 'float': 'left', 'margin-left': '40px'}
        ),
        html.Div(
            id='sag_div',
            children=[
                dcc.Upload(
                    id='upload-xml',
                    children=html.Div(['Drag and Drop or ', html.A('Select a File')]),
                    style={
                        'width': '100%',
                        'height': '60px',
                        'lineHeight': '60px',
                        'borderWidth': '1px',
                        'borderStyle': 'dashed',
                        'borderRadius': '5px',
                        'textAlign': 'center',
                        'margin': '10px'
                    },
                    multiple=False
                ),
                html.Div(id='output-container', style={'margin-top': '20px'}),
                dcc.Interval(
                    id='interval-component',
                    interval=1000,  # in milliseconds
                    n_intervals=0
                )
            ],
            style={'width': '750px', 'height': '1488px', 'background-color': '#DDF3F6', 'float': 'right', 'margin-left': '40px'}
        )
    ],style={'width': '1350px', 'height': '1488px'}
)






def read_data(addr, client_socket):
    if client_socket is None:
        raise ValueError("Client socket is not initialized.")
    message = f"READ*{addr}"
    client_socket.sendall(message.encode())
    response = client_socket.recv(1024).decode()
    response_str_lists[addr].append(int(response))
    print(response_str_lists)
    print(f"Sending read message: {message}")
    

def send_read(client_socket):
    if client_socket is None:
        print("Client socket is not initialized.")
        return

    for addr, on in zip(response_str_lists.keys(), switch_states):
        if on:
            read_data(addr, client_socket)
        time.sleep(0.1)   



@app.callback(
    Output('output-state', 'children'),
    [Input('connect-button', 'n_clicks'), Input('disconnect-button', 'n_clicks')],
    [State('ip-input', 'value'), State('port-input', 'value')]
)
def send_command_to_server(connect_clicks, disconnect_clicks, ip_value, port_value):
    ctx = callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'connect-button' and connect_clicks is not None and connect_clicks > 0:
        if ip_value and port_value:
            threading.Thread(target=main, args=(ip_value, int(port_value))).start()
            return "Connection established."
        else:
            return "Please enter IP address and port number."
    elif button_id == 'disconnect-button' and disconnect_clicks is not None and disconnect_clicks > 0:
        if client_socket:
            client_socket.close()
        return "Connection terminated."
    return None

@app.callback(
    Output('output-container-2', 'children'),
    [Input('upload-xml-2', 'contents')],
    [State('upload-xml-2', 'filename'), State('upload-xml-2', 'last_modified')]
)
def update_output_sol(contents, filename, last_modified):
    if contents is None:
        return html.Div(["Please upload an XML file."])
    
    try:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        config = parse_xml_config(decoded.decode('utf-8'))
        
        print("Config:", config)  # Debugging için config'i yazdır

        widgets = []

        if 'BooleanSwitch' in config:
            widgets.extend([process_toggleSwitch_config(toggleSwitch_params) for toggleSwitch_params in config['BooleanSwitch']])
        else:
            print("BooleanSwitch not found in config")  # Debugging için mesaj yazdır

        if not widgets:
            return html.Div(["Invalid XML configuration. Please check the file."])

        # Tablo oluşturma:
        table_rows = []
        row = []
        for i, widget in enumerate(widgets):
            row.append(html.Td(widget,style={'padding': '20px'}))
            if (i + 1) % 4 == 0:
                table_rows.append(html.Tr(row))
                row = []
        
        # Eğer son satır eksikse onu da ekleyelim
        if row:
            table_rows.append(html.Tr(row))

        return html.Div([
            html.H3(f"Uploaded file: {filename}"),
            html.Table(table_rows),
            dcc.Interval(id='interval-component-2-sol', interval=1000, n_intervals=0)
        ])

    except ValueError as e:
        return html.Div([f"Configuration Error: {e}"])
    except Exception as e:
        import traceback
        print(traceback.format_exc())  # Ayrıntılı hata mesajı yazdır
        return html.Div([f"An error occurred: {e}"])

@app.callback(
    Output('output-container', 'children'),
    [Input('upload-xml', 'contents')],
    [State('upload-xml', 'filename'), State('upload-xml', 'last_modified')]
)
def update_output(contents, filename, last_modified):
    if contents is None:
        return html.Div(["Please upload an XML file."])
    
    try:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        config = parse_xml_config(decoded.decode('utf-8'))
        print("Config:", config)
        
        widgets = []

        if 'Gauge' in config:
            widgets.extend([process_gauge_config(gauge_params) for gauge_params in config['Gauge']])
        
        if 'Thermometer' in config:
            widgets.extend([process_thermometer_config(thermometer_params) for thermometer_params in config['Thermometer']])

        if 'Graph' in config:
            widgets.extend(process_graph_config(config['Graph']))

        if not widgets:
            return html.Div(["Invalid XML configuration. Please check the file."])

        return html.Div([
            html.H3(f"Uploaded file: {filename}"),
            *widgets,
            dcc.Interval(id='interval-component-2', interval=1000, n_intervals=0)
        ])

    except ValueError as e:
        return html.Div([f"Configuration Error: {e}"])
    except Exception as e:
        return html.Div([f"An error occurred: {e}"])

@app.callback(
    Output('my-gauge', 'value'),
    [Input('interval-component-2', 'n_intervals')]
)
def update_gauge_value(n):
    if not response_str_lists[23]:
        return 0  # Eğer response_str_lists[23] boş ise, 0 değerini döndür
    else:
        return float(response_str_lists[23][-1])  # response_str_lists[23] boş değilse, son elemanını dönüştürüp döndür
@app.callback(
    Output('my-thermometer', 'value'),
    [Input('interval-component-2', 'n_intervals')]
)
def update_gauge_value(n):
    if not response_str_lists[24]:
        return 0  # Eğer response_str_lists[23] boş ise, 0 değerini döndür
    else:
        return float(response_str_lists[24][-1])  # response_str_lists[23] boş değilse, son elemanını dönüştürüp döndür
@app.callback(
    [Output(f'READ*{i}', 'on') for i in range(23, 31)],
    [Input(f'READ*{i}', 'on') for i in range(23, 31)],
)
def switch_callback(*args):
    global switch_states
    switch_states = list(args)
    print(f"Switch states updated: {switch_states}")
    while True:
        send_read(client_socket)
        
    return switch_states
@app.callback(
    Output('graph1', 'figure'),
    [Input('interval-component-2-sol', 'n_intervals')]
)
def update_indicator_graph_2(n):
    global X
    global Y_2
    if response_str_lists[25]:  # Eğer response_str_list boş değilse devam et
        X.append(X[-1] + 1)
        Y_2.append(response_str_lists[25][-1])
        trace = go.Scatter(
            x=list(X),
            y=list(Y_2),
            name='temperature_value',
            mode='lines+markers'
        )
        layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                           yaxis=dict(range=[0, 60]))
        return {'data': [trace], 'layout': layout}
    else:  # response_str_list boşsa grafik güncellemesini engelle
        raise dash.exceptions.PreventUpdate


@app.callback(
    Output('graph2', 'figure'),
    [Input('interval-component-2-sol', 'n_intervals')]
)
def update_indicator_graph_3(n):
    global X
    global Y_1
    if response_str_lists[26]:  # Eğer response_str_list boş değilse devam et
        X.append(X[-1] + 1)
        Y_1.append(response_str_lists[26][-1])
        trace2 = go.Scatter(
            x=list(X),
            y=list(Y_1),
            name='temperature_value',
            mode='lines+markers'
        )
        layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                           yaxis=dict(range=[0, 60]))
        return {'data': [trace2], 'layout': layout}
    else:  # response_str_list boşsa grafik güncellemesini engelle
        raise dash.exceptions.PreventUpdate



if __name__ == '__main__':
    app.run_server(debug=True, port=8312)


Connection established.
Config: {'BooleanSwitch': [{'id': 'READ*23', 'label': 'READ*23', 'labelPosition': 'top'}, {'id': 'READ*24', 'label': 'READ*24', 'labelPosition': 'top'}, {'id': 'READ*25', 'label': 'READ*25', 'labelPosition': 'top'}, {'id': 'READ*26', 'label': 'READ*26', 'labelPosition': 'top'}, {'id': 'READ*27', 'on': 'False', 'label': 'READ*27', 'labelPosition': 'top'}, {'id': 'READ*28', 'label': 'READ*28', 'labelPosition': 'top'}, {'id': 'READ*29', 'label': 'READ*29', 'labelPosition': 'top'}, {'id': 'READ*30', 'label': 'READ*30', 'labelPosition': 'top'}]}
Switch states updated: [False, False, False, False, False, False, False, False]
Switch states updated: [False, False, True, False, False, False, False, False]
{23: [], 24: [], 25: [62], 26: [], 27: [], 28: [], 29: [], 30: []}
Sending read message: READ*25
{23: [], 24: [], 25: [62, 62], 26: [], 27: [], 28: [], 29: [], 30: []}
Sending read message: READ*25
{23: [], 24: [], 25: [62, 62, 43], 26: [], 27: [], 28: [], 29: [], 30: [